In [1]:
import os
import sys
import warnings
warnings.filterwarnings('ignore')

import numpy as np
from scipy import stats, signal
import scipy.io as sio

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold, LeaveOneOut
from sklearn import preprocessing, model_selection, metrics
from sklearn.metrics import accuracy_score, confusion_matrix

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import pandas as pd

%matplotlib notebook

In [2]:
def load_vu_all_files(filedirectory, numfiles, delaybool):
# Get data from .mat files
# Assumes input data are in the format of time (rows) x channels (columns)
    if len(numfiles) == 0:
        numfiles_start = 0
        numfiles_end = len(os.listdir(filedirectory)) # Use all files in directory
        print('Loading {} processed data file(s) ...'.format(numfiles_end - numfiles_start + 1))     
    else:
        numfiles_start = numfiles[0]
        numfiles_end = numfiles[1]
        print('Loading {} processed data file(s) ...'.format(numfiles_end - numfiles_start + 1))      

    filedict = {}
    allindtrigsdict = {}
        
    for fileindex, filename in enumerate(sorted(os.listdir(filedirectory))):
        if numfiles_start <= fileindex < numfiles_end:
            print('=== {} ==='.format(filename))
            mat_contents = sio.loadmat(filedirectory + '\\' + filename)
            daqdata = mat_contents['python_input']
            
            if fileindex == numfiles_start:
                chanlabels = mat_contents['chanlabels'].flatten()
                allcolheaders = []
                for i in np.arange(len(chanlabels)):
                    allcolheaders.append(chanlabels[i][0])
                        
            # Get post-processed channel data
            chans = daqdata[:,:31]
            locomode = daqdata[:,31]
            
            if delaybool == True: # 3 frame (~90 ms) delay
                use_trig = daqdata[:,33]
                # Define valid triggers (5-digit)
#                 hc_sm_trigs = [24210, 24510, 54510, 13410, 21410, 54210, 24610, 24410, 64610, 64210, 44410, 44210]
                hc_sm_trigs = [24210, 24510, 54510, 21410, 54210, 24610, 24410, 64610, 64210, 44410, 44210]
                mst_sm_trigs = [41410, 41420, 41260]
#                 to_sm_trigs = [22230, 21130, 52530, 62630, 41430, 42430, 21330, 22370, 11330, 22330]
                to_sm_trigs = [22230, 21130, 52530, 62630, 41430, 42430, 21330, 22370, 22330]
                msw_sm_trigs = [39340, 39280]
#                 shc_sm_trigs = [13110, 24410, 13410, 21410]
                shc_sm_trigs = [13110, 13410]
#                 sto_sm_trigs = [11130, 21130, 21330, 11330]
                sto_sm_trigs = [11130, 11330]
            else: # No delay
                use_trig = daqdata[:,32]
                # Define valid triggers (4-digit)
#                 hc_sm_trigs = [2421, 2451, 5451, 1341, 2141, 5421, 2461, 2441, 6461, 6421, 4441, 4421]
                hc_sm_trigs = [2421, 2451, 5451, 2141, 5421, 2461, 2441, 6461, 6421, 4441, 4421]
                mst_sm_trigs = [4141, 4142, 4126]
#                 to_sm_trigs = [2223, 2113, 5253, 6263, 4143, 4243, 2133, 2237, 1133, 2233]
                to_sm_trigs = [2223, 2113, 5253, 6263, 4143, 4243, 2133, 2237, 2233]
                msw_sm_trigs = [3934, 3928]
#                 shc_sm_trigs = [1311, 2441, 1341, 2141]
                shc_sm_trigs = [1311, 1341]
#                 sto_sm_trigs = [1113, 2113, 2133, 1133]
                sto_sm_trigs = [1113, 1133]

            trig_diff = np.diff(use_trig)
            trig_ind = np.where(trig_diff > 0)[0] + 1
            
            hc_ind, mst_ind, to_ind, msw_ind, shc_ind, sto_ind = [], [], [], [], [], []
            hc_trig, mst_trig, to_trig, msw_trig, shc_trig, sto_trig = [], [], [], [], [], []
            
            for i in np.arange(len(trig_ind)):
                if use_trig[trig_ind[i]] in hc_sm_trigs:
                    hc_ind.append(int(trig_ind[i]))
                    hc_trig.append(int(use_trig[trig_ind[i]]))
                if use_trig[trig_ind[i]] in mst_sm_trigs:
                    mst_ind.append(int(trig_ind[i]))
                    mst_trig.append(int(use_trig[trig_ind[i]]))
                if use_trig[trig_ind[i]] in to_sm_trigs:
                    to_ind.append(int(trig_ind[i]))
                    to_trig.append(int(use_trig[trig_ind[i]]))
                if use_trig[trig_ind[i]] in msw_sm_trigs:
                    msw_ind.append(int(trig_ind[i]))
                    msw_trig.append(int(use_trig[trig_ind[i]]))
                if use_trig[trig_ind[i]] in shc_sm_trigs:
                    shc_ind.append(int(trig_ind[i]))
                    shc_trig.append(int(use_trig[trig_ind[i]]))
                if use_trig[trig_ind[i]] in sto_sm_trigs:
                    sto_ind.append(int(trig_ind[i]))
                    sto_trig.append(int(use_trig[trig_ind[i]]))
            
            hc_ind, hc_trig = np.array(hc_ind), np.array(hc_trig)
            mst_ind, mst_trig = np.array(mst_ind), np.array(mst_trig)
            to_ind, to_trig = np.array(to_ind), np.array(to_trig)
            msw_ind, msw_trig = np.array(msw_ind), np.array(msw_trig)
            shc_ind, shc_trig = np.array(shc_ind), np.array(shc_trig)
            sto_ind, sto_trig = np.array(sto_ind), np.array(sto_trig)
            
            # Save the post-processed data and indices/triggers into dictionaries
            filedict[filename] = [chans,locomode,daqdata[:,30],daqdata[:,31]]
            allindtrigsdict[filename] = {'HC': np.vstack((hc_ind,hc_trig)).T, 
                                         'MST': np.vstack((mst_ind,mst_trig)).T, 
                                         'TO': np.vstack((to_ind,to_trig)).T, 
                                         'MSW': np.vstack((msw_ind,msw_trig)).T,
                                         'SHC': np.vstack((shc_ind,shc_trig)).T,
                                         'STO': np.vstack((sto_ind,sto_trig)).T}
            
    # filedict has the filename as the key and the 2D DAQ data and targets as entries
    # allcolheaders has the channel names
    print('Finished!')
    
    return filedict, allindtrigsdict, allcolheaders


def unpack_files(filedict, allindtrigsdict, filekeys, arginput, allcolheaders):    
    # Unpack the list of arguments (arginput)
    # Windowing parameters
    
    TRAIN_SIZE = arginput[0] # Sliding window length
    
    # Train/test split parameters
    FOR_TEST = arginput[1]
    TOTAL_FILES = len(filekeys)
    if len(FOR_TEST) > 1 or type(FOR_TEST[0]) is str:
        print('Generating train/test data from {} specified file(s):'.format(len(FOR_TEST)))
        TEST_FILE_INDS = []
        for testfileind, testfile in enumerate(FOR_TEST):
            TEST_FILE_INDS.append(filekeys.index(testfile))
        TEST_FILE_INDS = np.array(TEST_FILE_INDS)
        TRAIN_FILE_INDS = np.setdiff1d(np.arange(TOTAL_FILES),TEST_FILE_INDS)
    else:
        print('Randomly select train/test data based on testing data proportion ({}%):'.format(int(100*FOR_TEST[0])))
        TOTAL_FOLDS = int(1/(FOR_TEST[0]))
        SPLIT_FILES = np.array_split(np.arange(TOTAL_FILES),TOTAL_FOLDS)
        TEST_FILE_INDS = SPLIT_FILES[np.random.randint(0,TOTAL_FOLDS-1)]
        TRAIN_FILE_INDS = np.setdiff1d(np.arange(TOTAL_FILES),TEST_FILE_INDS)
    print()
    
    # Channels to use
    CHAN_MECH = arginput[2]
    
    # Printing parameters
    PRINT_SUMMARY = arginput[3]
    
    alldict = {}
    alldict['Combined File Index'] = []
        
    for gaitevent in ['HC', 'MST', 'TO', 'MSW', 'SHC', 'STO']:
        alldict['Combined ' + gaitevent + ' Windows'] = []
        alldict['Combined ' + gaitevent + ' Features'] = []
        alldict['Combined ' + gaitevent + ' Triggers'] = [] 
        alldict['Combined ' + gaitevent + ' File Index'] = []
        
    for fkeyind, fkey in enumerate(filekeys):
        print('Preparing file {}: {} ...'.format(fkeyind+1,fkey))

        data = filedict[fkey][0]
        locomode = filedict[fkey][1]
        indtrigs = allindtrigsdict[fkey]

        alleventdict, featnames = event_windows(indtrigs, data, TRAIN_SIZE, CHAN_MECH, allcolheaders)
        
        alleventfeats = {'HC': alleventdict['HC'][0], 
                         'MST': alleventdict['MST'][0], 
                         'TO': alleventdict['TO'][0], 
                         'MSW': alleventdict['MSW'][0],
                         'SHC': alleventdict['SHC'][0],
                         'STO': alleventdict['STO'][0]}

        for gaitevent in ['HC', 'MST', 'TO', 'MSW', 'SHC', 'STO']:
            # Save the features extracted from each gait event
            alldict['Combined ' + gaitevent + ' Features'].append(alleventfeats[gaitevent])
            # Save the trigger associated with each gait event
            alldict['Combined ' + gaitevent + ' Triggers'].append(alleventdict[gaitevent][1])
            # Save file index associated with each gait event
            alldict['Combined ' + gaitevent + ' File Index'].append(np.tile(fkeyind,len(alleventdict[gaitevent][1])))
    print()
    
    for gaitevent in ['HC', 'MST', 'TO', 'MSW', 'SHC', 'STO']:
        # Remove empty lists
        alldict['Combined ' + gaitevent + ' Features'] = [features for features in alldict['Combined ' + gaitevent + ' Features'] if len(features) > 0]
        alldict['Combined ' + gaitevent + ' Triggers'] = [triggers for triggers in alldict['Combined ' + gaitevent + ' Triggers'] if len(triggers) > 0]
        alldict['Combined ' + gaitevent + ' File Index'] = [fileinds for fileinds in alldict['Combined ' + gaitevent + ' File Index'] if len(fileinds) > 0]
        
        alldict['Combined ' + gaitevent + ' Features'] = np.concatenate(alldict['Combined ' + gaitevent + ' Features'])
        alldict['Combined ' + gaitevent + ' Triggers'] = np.concatenate(alldict['Combined ' + gaitevent + ' Triggers'])
        alldict['Combined ' + gaitevent + ' File Index'] = np.concatenate(alldict['Combined ' + gaitevent + ' File Index'])
        
    if PRINT_SUMMARY:
        print('Aggregated HC feature dimensions: {}'.format(alldict['Combined HC Features'].shape))
        print('Aggregated MST feature dimensions: {}'.format(alldict['Combined MST Features'].shape))
        print('Aggregated TO feature dimensions: {}'.format(alldict['Combined TO Features'].shape))
        print('Aggregated MSW feature dimensions: {}'.format(alldict['Combined MSW Features'].shape))
        print('Aggregated SHC feature dimensions: {}'.format(alldict['Combined SHC Features'].shape))
        print('Aggregated STO feature dimensions: {}'.format(alldict['Combined STO Features'].shape))
        print()
        
    return alldict, featnames, TRAIN_FILE_INDS, TEST_FILE_INDS


def event_windows(indtrigs, data, train_window, chanmech, allcolheaders):
# Get data from windows near gait events specified by indtrigs
# Try different sized windows (pre_stance before stance and pre_swing before swing)
# Try data augmentation (get aug_windows_per_event extra windows beginning aug_pre before to aug_post after the gait event)
# scale = normalize data (boolean)

    mechheaders = [allcolheaders[i] for i in chanmech]
    eventkeys = list(indtrigs.keys())
        
    alleventdict = {}
    for eventkeyind, eventkey in enumerate(eventkeys):
        allfeats, trig_list = [], []
        
        inds = indtrigs[eventkey][:,0]
        trigs = indtrigs[eventkey][:,1]
        
        # Remove triggers occurring less than 300 ms into DAQ file
        keepinds = [inds > train_window]
        inds = inds[keepinds]
        trigs = trigs[keepinds] 
        
        if len(inds) > 0:
            for i in range(inds.shape[0]): # Iterates over indices
                x_i = data[(inds[i]-train_window):inds[i],:]
                y_i = trigs[i]

                feats, featslabels = feats_from_window(x_i[-300:],chanmech,mechheaders)

                allfeats.append(feats)
                trig_list.append(y_i)

            # Convert from list of arrays to 3D array        
            alleventdict[eventkey] = [np.array(allfeats)] # Extracted features
            alleventdict[eventkey].append(np.array(trig_list)) # Target
        else:
            print('No ' + eventkey + ' events...')
            alleventdict[eventkey] = [[],[]]
            
    # alleventdict has the gait event (HC, TO) as the key and contains the 3d array of the extracted windows (300 ms for feature extraction and pre_stance, pre_swing) and targets
    return alleventdict, featslabels


def feats_from_window(window,chanmech,mechheaders):
    if len(chanmech) > 0:
        mechfeats, mechfeatsnames = getmechfeats(window,chanmech,mechheaders)
    else:
        mechfeats = []

    feats = mechfeats.T    
    featslabels = mechfeatsnames
    
    return feats, featslabels


def getmechfeats(X,chanmech,mechheaders):    
    X_mech = X[:,chanmech]

    X_min = np.min(X_mech,axis=0)
    X_max = np.max(X_mech,axis=0)
    X_mean = np.mean(X_mech,axis=0)
    X_std = np.std(X_mech,axis=0)
    X_init = X_mech[0]
    X_final = X_mech[-1]

    mechfeats = np.array([X_min,X_max,X_init,X_final,X_mean,X_std]).flatten()
    mechfeatsnames = []
    
    featsnames = [' Min',' Max',' Initial',' Final',' Mean',' SD']
    
    for names in featsnames:
        for mechchan in mechheaders:
            mechfeatsnames.append(mechchan + names)

    return mechfeats, mechfeatsnames

In [19]:
def unpack_trig(triggers):
# Get the first and third digits of the four-digit trigger in order to make mode-specific classifiers

    trigstr = triggers.astype(int).astype(str)
    leavemode = np.array([int(trig[0]) for trig in trigstr])
    entermode = np.array([int(trig[2]) for trig in trigstr])
    steptype = np.array([int(leavemode[i] == entermode[i]) for i in range(len(leavemode))])
    
    return leavemode, entermode, steptype
    
    
def split_by_leavemode(feat_data, leavemode, useleave, entermode, steptype, fileind):
# Organize data into dictionary with two-layer keys: gait event (RHC, RTO, LHC, LTO) and mode (LW, RA, RD, SA, SD)
# DS_FACTOR = down-sampling factor (may improve NN performance by reducing dimensionality of raw data)
# fileind keeps track of which file each gait event came from (in order to do leave-one-circuit-out cross validation)
    # 1 = St, 2 = LW, 3 = SA, 4 = SD, 5 = RA, 6 = RD
    
    lmode_inds = [ind for ind in range(len(leavemode)) if leavemode[ind] in useleave] 
    feat_data_inds = feat_data[lmode_inds]
    target_inds = entermode[lmode_inds]
    file_inds = fileind[lmode_inds]
    type_bin_inds = steptype[lmode_inds]
    
    # Merge RA targets with LW targets
    target_inds[target_inds == 5] = 2

    data_ms = [feat_data_inds, target_inds, file_inds, type_bin_inds]
    
    return data_ms 
 

def make_modespec(alldict,USE_FEAT):
# Make modespec_dict to organize all events/modes
    
    hc_leave, hc_enter, hc_type = unpack_trig(alldict['Combined HC Triggers'])
    mst_leave, mst_enter, mst_type = unpack_trig(alldict['Combined MST Triggers'])
    to_leave, to_enter, to_type = unpack_trig(alldict['Combined TO Triggers'])
    msw_leave, msw_enter, msw_type = unpack_trig(alldict['Combined MSW Triggers'])
    shc_leave, shc_enter, shc_type = unpack_trig(alldict['Combined SHC Triggers'])
    sto_leave, sto_enter, sto_type = unpack_trig(alldict['Combined STO Triggers'])
    
    hc_lw_ms = split_by_leavemode(alldict['Combined HC Features'][:,USE_FEAT], hc_leave, [2,5], hc_enter, hc_type, alldict['Combined HC File Index'])
    hc_rd_ms = split_by_leavemode(alldict['Combined HC Features'][:,USE_FEAT], hc_leave, [6], hc_enter, hc_type, alldict['Combined HC File Index'])
    hc_sd_ms = split_by_leavemode(alldict['Combined HC Features'][:,USE_FEAT], hc_leave, [4], hc_enter, hc_type, alldict['Combined HC File Index'])
    
    mst_sd_ms = split_by_leavemode(alldict['Combined MST Features'][:,USE_FEAT], mst_leave, [4], mst_enter, mst_type, alldict['Combined MST File Index'])
#     to_ms = split_by_leavemode(alldict['Combined TO Features'][:,USE_FEAT], to_leave, [1,2,4,5,6], to_enter, to_type, alldict['Combined TO File Index'])
    to_ms = split_by_leavemode(alldict['Combined TO Features'][:,USE_FEAT], to_leave, [2,4,5,6], to_enter, to_type, alldict['Combined TO File Index'])
    msw_sa_ms = split_by_leavemode(alldict['Combined MSW Features'][:,USE_FEAT], msw_leave, [3], msw_enter, msw_type, alldict['Combined MSW File Index'])
#     shc_ms = split_by_leavemode(alldict['Combined SHC Features'][:,USE_FEAT], shc_leave, [1,2], shc_enter, shc_type, alldict['Combined SHC File Index'])
#     sto_ms = split_by_leavemode(alldict['Combined STO Features'][:,USE_FEAT], sto_leave, [1,2], sto_enter, sto_type, alldict['Combined STO File Index'])
    shc_ms = split_by_leavemode(alldict['Combined SHC Features'][:,USE_FEAT], shc_leave, [1], shc_enter, shc_type, alldict['Combined SHC File Index'])
    sto_ms = split_by_leavemode(alldict['Combined STO Features'][:,USE_FEAT], sto_leave, [1], sto_enter, sto_type, alldict['Combined STO File Index'])
    
    modespec_dict = {'HC_LW': hc_lw_ms, 'HC_RD': hc_rd_ms, 'HC_SD': hc_sd_ms,
                     'MST_SD': mst_sd_ms, 'TO': to_ms, 'MSW_SA': msw_sa_ms, 'SHC': shc_ms, 'STO': sto_ms}
    
    return modespec_dict    


def lda_classify(train_in,train_targ,test_in,test_targ,pca_proportion):
    lda = LinearDiscriminantAnalysis()
    
    train_in_scaler = preprocessing.StandardScaler().fit(train_in)
    train_in_scaled = train_in_scaler.transform(train_in)
    test_in_scaled = train_in_scaler.transform(test_in)

    train_in_scaler_pca = PCA().fit(train_in_scaled)
    train_in_scaled_pca_xfm = train_in_scaler_pca.transform(train_in_scaled)
    test_in_scaled_pca_xfm = train_in_scaler_pca.transform(test_in_scaled)

    pcaexplainedvar = np.cumsum(train_in_scaler_pca.explained_variance_ratio_)
    if pca_proportion < 1:
        pcanumcomps = min(min(np.where(pcaexplainedvar > pca_proportion))) + 1
    else:
        pcanumcomps = pca_proportion

    unique_modes = np.unique(train_targ)
    lda.set_params(priors = np.ones(len(unique_modes))/len(unique_modes))

    test_pred = lda.fit(train_in_scaled_pca_xfm[:,0:pcanumcomps],train_targ).predict(test_in_scaled_pca_xfm[:,0:pcanumcomps])
    test_pred_prob = lda.fit(train_in_scaled_pca_xfm[:,0:pcanumcomps],train_targ).predict_proba(test_in_scaled_pca_xfm[:,0:pcanumcomps])
    test_gtruth = test_targ
    
    return test_pred, test_gtruth, test_pred_prob

    
def ms_classify_results(msdict,pcaprop,trainfiles,testfiles):
    ohe = preprocessing.OneHotEncoder()
    lda = LinearDiscriminantAnalysis()
    kf = KFold(n_splits=10, shuffle=True)
    loo = LeaveOneOut()
    
    # Define histogram labels
    gaitmodes = ['St','LW','SA','SD','RA','RD']
    
    all_pred, all_gtruth, all_type, all_files = [], [], [], []
    
    for classifiers in list(allevents_ms.keys()):
        print('Classifier: {}'.format(classifiers))
        
        event_mode_features = msdict[classifiers][0]
        event_mode_targets = msdict[classifiers][1]
        event_mode_files = msdict[classifiers][2]
        event_mode_typebin = msdict[classifiers][3]
    
        print('Unique: {}'.format(np.unique(event_mode_targets)))
        print(gaitmodes)
        print(np.histogram(event_mode_targets,bins=[0.5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5])[0])  
        
        if len(np.unique(event_mode_targets)) > 1:
            event_mode_pred = np.zeros((len(event_mode_targets),1))
            if len(trainfiles) > 0 and len(testfiles) > 0:
                train_inds = [i for i in range(len(event_mode_files)) if event_mode_files[i] in trainfiles]
                test_inds = [i for i in range(len(event_mode_files)) if event_mode_files[i] in testfiles]
                pred, gtruth, pred_prob = lda_classify(event_mode_features[train_inds],event_mode_targets[train_inds],event_mode_features[test_inds],event_mode_targets[test_inds],pcaprop)           
                event_mode_pred[test_inds] = pred
            else:
                for train_inds, test_inds in loo.split(event_mode_features,event_mode_targets):
                    pred, gtruth, pred_prob = lda_classify(event_mode_features[train_inds],event_mode_targets[train_inds],event_mode_features[test_inds],event_mode_targets[test_inds],pcaprop)           
                    event_mode_pred[test_inds] = pred

            print('LOO Accuracy: {}'.format(accuracy_score(event_mode_pred,event_mode_targets)))
            print()
            
            all_pred.append(event_mode_pred)
            all_gtruth.append(event_mode_targets)
            all_type.append(event_mode_typebin)
            all_files.append(event_mode_files) 
        else:
            print('Skipping {} because only one class'.format(classifiers))
            print()
    
    all_pred = np.concatenate(all_pred)
    all_gtruth = np.concatenate(all_gtruth)
    all_type = np.concatenate(all_type)
    all_files = np.concatenate(all_files)
    
    return all_pred, all_gtruth, all_type, all_files

In [4]:
def select_chan(USE_MECH_CHAN_NAME,allcolheaders,allfeatnames):
    for featind, featname in enumerate(allfeatnames):
        if '_' in featname:
            allfeatnames[featind] = featname.replace('_',' ')
    
    for colind, colheader in enumerate(allcolheaders):
        if '_' in colheader:
            allcolheaders[colind] = colheader.replace('_',' ')
    
    if len(USE_MECH_CHAN_NAME) > 0:
        USE_MECH_CHAN = []
        USE_MECH_FEAT = []
        for channameind, channame in enumerate(USE_MECH_CHAN_NAME): 
            MECH_CHAN_IND = [allcolheaders.index(colname) for colname in allcolheaders if channame in colname]
            MECH_FEAT_IND = [allfeatnames.index(colname) for colname in allfeatnames if channame in colname]
            USE_MECH_CHAN.append(MECH_CHAN_IND)
            USE_MECH_FEAT.append(MECH_FEAT_IND)
        
        USE_MECH_CHAN = np.array(sorted(sum(USE_MECH_CHAN,[])),dtype=int)
        USE_MECH_FEAT = np.array(sorted(sum(USE_MECH_FEAT,[])),dtype=int)

        USE_CHAN = USE_MECH_CHAN
        USE_FEAT = USE_MECH_FEAT
    else:
        USE_CHAN = np.arange(len(allcolheaders))
        USE_FEAT = np.arange(len(allfeatnames))
    
    return np.unique(USE_CHAN), np.unique(USE_FEAT)

### Load and organize raw data

In [14]:
# Just get the filenames from the filedir
# FILEDIR = u'C:\\Users\\bhu\\Git\\Bilateral_Intent_Recognition\\Data\\Prosthesis'
FILEDIR = u'Z:\\Lab Member Folders\\Blair Hu\\Contralateral Prosthesis Control 2017\\TF01_Goldie04_121917_Dev\\DATA\\FORPY'
NUM_FILES = [] # if empty list, load all files; otherwise [X] means load X files
DELAYED = False
filedict, allindtrigsdict, allcolheaders = load_vu_all_files(FILEDIR, NUM_FILES, DELAYED)
filekeys = sorted(list(filedict.keys()))

Loading 127 processed data file(s) ...
=== circuit_even_121917_data_001_pydaq.mat ===
=== circuit_even_121917_data_002_pydaq.mat ===
=== circuit_even_121917_data_003_pydaq.mat ===
=== circuit_even_121917_data_004_pydaq.mat ===
=== circuit_even_121917_data_007_pydaq.mat ===
=== circuit_even_121917_data_009_pydaq.mat ===
=== circuit_even_121917_data_010_pydaq.mat ===
=== circuit_even_121917_data_012_pydaq.mat ===
=== circuit_even_121917_data_013_pydaq.mat ===
=== circuit_even_121917_data_014_pydaq.mat ===
=== circuit_even_121917_data_015_pydaq.mat ===
=== circuit_even_121917_data_017_pydaq.mat ===
=== circuit_even_121917_data_018_pydaq.mat ===
=== circuit_even_121917_data_020_pydaq.mat ===
=== circuit_even_121917_data_022_pydaq.mat ===
=== circuit_even_121917_data_023_pydaq.mat ===
=== circuit_even_121917_data_025_pydaq.mat ===
=== circuit_even_121917_data_026_pydaq.mat ===
=== circuit_even_121917_data_028_pydaq.mat ===
=== circuit_even_121917_data_030_pydaq.mat ===
=== circuit_even_1219

In [15]:
TRAIN_SIZE = 300 # Training windows are 300 time-steps long for sliding windows
# FOR_TEST = ['circuit_even_data_020.mat','circuit_odd_data_019.mat'] # Filenames to use for testing
FOR_TEST = [0.05]
CHAN_MECH = np.arange(31)
PRINT_SUMMARY = True # Print dimensions of aggregated data from all files

arginput = [TRAIN_SIZE,FOR_TEST,CHAN_MECH,PRINT_SUMMARY]

alldict, allfeatnames, _, _ = unpack_files(filedict,allindtrigsdict,filekeys,arginput,allcolheaders)

Randomly select train/test data based on testing data proportion (5%):

Preparing file 1: circuit_even_121917_data_001_pydaq.mat ...
No MST events...
Preparing file 2: circuit_even_121917_data_002_pydaq.mat ...
No MSW events...
No MST events...
No STO events...
Preparing file 3: circuit_even_121917_data_003_pydaq.mat ...
No MSW events...
No MST events...
Preparing file 4: circuit_even_121917_data_004_pydaq.mat ...
No MSW events...
No MST events...
Preparing file 5: circuit_even_121917_data_007_pydaq.mat ...
No MST events...
Preparing file 6: circuit_even_121917_data_009_pydaq.mat ...
No MSW events...
No MST events...
Preparing file 7: circuit_even_121917_data_010_pydaq.mat ...
No MSW events...
No MST events...
Preparing file 8: circuit_even_121917_data_012_pydaq.mat ...
No MSW events...
Preparing file 9: circuit_even_121917_data_013_pydaq.mat ...
No MST events...
Preparing file 10: circuit_even_121917_data_014_pydaq.mat ...
No MSW events...
No SHC events...
No MST events...
No STO even

No MSW events...
No MST events...
No STO events...
Preparing file 117: walk_121917_data_006_pydaq.mat ...
No MSW events...
No MST events...
Preparing file 118: walk_121917_data_007_pydaq.mat ...
No MSW events...
No MST events...
Preparing file 119: walk_121917_data_008_pydaq.mat ...
No MSW events...
No MST events...
Preparing file 120: walk_121917_data_009_pydaq.mat ...
No TO events...
No HC events...
No MSW events...
No MST events...
Preparing file 121: walk_121917_data_010_pydaq.mat ...
No MSW events...
No MST events...
Preparing file 122: walk_121917_data_011_pydaq.mat ...
No MSW events...
No MST events...
Preparing file 123: walk_121917_data_012_pydaq.mat ...
No MSW events...
No MST events...
Preparing file 124: walk_121917_data_013_pydaq.mat ...
No MSW events...
No MST events...
Preparing file 125: walk_121917_data_014_pydaq.mat ...
No MSW events...
No MST events...
Preparing file 126: walk_121917_data_015_pydaq.mat ...
No MSW events...
No MST events...

Aggregated HC feature dime

### Select Channels

In [10]:
vu_only = list(range(17))
contra_thigh = list(range(17)) + list(range(23,29)) + [30]
contra_shank = list(range(23)) + [29]
contra_both = list(range(31))
chanuse = contra_both

USE_MECH_CHAN_NAME = [allcolheaders[chanuse[i]] for i in range(len(chanuse))]

USE_CHAN, USE_FEAT = select_chan(USE_MECH_CHAN_NAME,allcolheaders,allfeatnames)
print('=====Using the following {} channels:====='.format(len(USE_CHAN)))
for chan in USE_CHAN:
    print(allcolheaders[chan])
print()
print('=====Using the following {} features:====='.format(len(USE_FEAT)))
for feat in USE_FEAT:
    print(allfeatnames[feat])

=====Using the following 31 channels:=====
Knee Angle
Knee Vel
Knee Current
Ankle Angle
Ankle Vel
Ankle Current
VU Ax
VU Ay
VU Az
VU Gx
VU Gz
VU Gy
Shank Angle
Thigh Angle
Knee Ref
Ankle Ref
Load
Shank Ax
Shank Ay
Shank Az
Shank Gy
Shank Gz
Shank Gx
Thigh Ax
Thigh Ay
Thigh Az
Thigh Gy
Thigh Gz
Thigh Gx
Contra Shank
Contra Thigh

=====Using the following 186 features:=====
Knee Angle Min
Knee Vel Min
Knee Current Min
Ankle Angle Min
Ankle Vel Min
Ankle Current Min
VU Ax Min
VU Ay Min
VU Az Min
VU Gx Min
VU Gz Min
VU Gy Min
Shank Angle Min
Thigh Angle Min
Knee Ref Min
Ankle Ref Min
Load Min
Shank Ax Min
Shank Ay Min
Shank Az Min
Shank Gy Min
Shank Gz Min
Shank Gx Min
Thigh Ax Min
Thigh Ay Min
Thigh Az Min
Thigh Gy Min
Thigh Gz Min
Thigh Gx Min
Contra Shank Min
Contra Thigh Min
Knee Angle Max
Knee Vel Max
Knee Current Max
Ankle Angle Max
Ankle Vel Max
Ankle Current Max
VU Ax Max
VU Ay Max
VU Az Max
VU Gx Max
VU Gz Max
VU Gy Max
Shank Angle Max
Thigh Angle Max
Knee Ref Max
Ankle Ref Max
Lo

### Use PCA to visualize event-specific feature space

In [ ]:
event = 'MSW'

trainfiles = []
testfiles = []
traineventinds = [fileind for fileind in np.arange(len(alldict['Combined ' + event + ' File Index'])) if alldict['Combined ' + event + ' File Index'][fileind] in train_files]
testeventinds = [fileind for fileind in np.arange(len(alldict['Combined ' + event + ' File Index'])) if alldict['Combined ' + event + ' File Index'][fileind] in test_files]

leave, enter, steptype = unpack_trig(alldict['Combined ' + event + ' Triggers']) # Get the leaving and entering modes from the triggers

# Fit scaler with training files
trainevent_scaler = preprocessing.StandardScaler().fit(alldict['Combined ' + event + ' Features'][traineventinds][:,USE_FEAT])
trainevent_norm = trainevent_scaler.transform(alldict['Combined ' + event + ' Features'][traineventinds][:,USE_FEAT])
testevent_norm = trainevent_scaler.transform(alldict['Combined ' + event + ' Features'][testeventinds][:,USE_FEAT])

# Fit PCA with training files
trainevent_pca = PCA().fit(trainevent_norm)
trainevent_dimred = trainevent_pca.transform(trainevent_norm)
testevent_dimred = trainevent_pca.transform(testevent_norm)   

pcolor = {1: 'k', 2: 'b', 3:'c', 4:'m', 5:'g', 6:'r'}
    
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
for wind in np.arange(0,np.shape(trainevent_dimred)[0]):
    ax.scatter(trainevent_dimred[wind,0],trainevent_dimred[wind,1],trainevent_dimred[wind,2],color=pcolor[enter[traineventinds][wind]],alpha=0.3)

### Prepare Mode Specific Classification

In [20]:
# Re-organize data for mode-specific classifiers for each gait event (downsampling optional)
allevents_ms = make_modespec(alldict, USE_FEAT)

### Baseline Mode Specific Classification

In [21]:
pcaprop = 50
print('MS classification:')
all_pred, all_gtruth, all_type, all_files = ms_classify_results(allevents_ms,pcaprop,[],[])

MS classification:
Classifier: HC_LW
Unique: [2 4 6]
['St', 'LW', 'SA', 'SD', 'RA', 'RD']
[  0 599   0  10   0  23]
LOO Accuracy: 0.9936708860759493

Classifier: MST_SD
Unique: [2 4]
['St', 'LW', 'SA', 'SD', 'RA', 'RD']
[ 0 11  0 60  0  0]
LOO Accuracy: 1.0

Classifier: STO
Unique: [1 3]
['St', 'LW', 'SA', 'SD', 'RA', 'RD']
[639   0  82   0   0   0]
LOO Accuracy: 1.0

Classifier: MSW_SA
Unique: [2 3]
['St', 'LW', 'SA', 'SD', 'RA', 'RD']
[ 0 13 76  0  0  0]
LOO Accuracy: 0.9775280898876404

Classifier: HC_RD
Unique: [2 6]
['St', 'LW', 'SA', 'SD', 'RA', 'RD']
[ 0 24  0  0  0 85]
LOO Accuracy: 0.981651376146789

Classifier: SHC
Unique: [1 4]
['St', 'LW', 'SA', 'SD', 'RA', 'RD']
[766   0   0  21   0   0]
LOO Accuracy: 1.0

Classifier: HC_SD
Unique: [2 4]
['St', 'LW', 'SA', 'SD', 'RA', 'RD']
[ 0 20  0 40  0  0]
LOO Accuracy: 0.9833333333333333

Classifier: TO
Unique: [1 2 3 4 6]
['St', 'LW', 'SA', 'SD', 'RA', 'RD']
[160 621   7  60   0 109]
LOO Accuracy: 0.9738766980146291



In [22]:
cm = confusion_matrix(all_pred,all_gtruth)
class_totals = cm.sum(axis=1)
norm_cm = np.zeros((5,5))
for i in range(5):
    norm_cm[i,:] = cm[i,:]/class_totals[i]

print(pd.DataFrame(100*norm_cm))

t_step = np.where(all_type == 0)[0]
ss_step = np.where(all_type == 1)[0]

print('\nSS Accuracy: {}'.format(accuracy_score(all_pred[ss_step],all_gtruth[ss_step])))
print('Trans Accuracy: {}'.format(accuracy_score(all_pred[t_step],all_gtruth[t_step])))
print('Overall Accuracy: {}'.format(accuracy_score(all_pred,all_gtruth)))

print('\nSS steps: {}'.format(len(ss_step)))
print('T steps: {}'.format(len(t_step)))

print('\nFiles with errors:')
for fnum in np.unique([all_files[i] for i in range(len(all_files)) if all_pred[i] != all_gtruth[i]]):
    print(filekeys[fnum])

           0          1          2          3          4
0  98.551637   1.385390   0.000000   0.000000   0.062972
1   0.000000  99.683544   0.079114   0.000000   0.237342
2   0.000000   1.204819  98.795181   0.000000   0.000000
3   0.000000   1.030928   0.000000  98.453608   0.515464
4   0.000000   0.934579   0.000000   0.000000  99.065421

SS Accuracy: 0.9910743801652893
Trans Accuracy: 0.9825436408977556
Overall Accuracy: 0.9900758902510216

SS steps: 3025
T steps: 401

Files with errors:
circuit_even_121917_data_001_pydaq.mat
circuit_even_121917_data_002_pydaq.mat
circuit_even_121917_data_009_pydaq.mat
circuit_even_121917_data_013_pydaq.mat
circuit_even_121917_data_015_pydaq.mat
circuit_even_121917_data_018_pydaq.mat
circuit_even_121917_data_022_pydaq.mat
circuit_even_121917_data_026_pydaq.mat
circuit_even_121917_data_035_pydaq.mat
circuit_even_121917_data_048_pydaq.mat
circuit_even_121917_data_050_pydaq.mat
circuit_even_121917_data_051_pydaq.mat
circuit_odd_121917_data_003_pydaq.ma